Adding the distances to every OD pair. I only could use the distances from the real timetable and look up the distances name by name. That is the reason I should create a matrix to get the real distances between not neighboring stations.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.concat(dfs, axis=0).reset_index(drop=True)

In [ ]:
stations = df.groupby('From').count()
stations

In [ ]:
stations_to = df.groupby('To').count()
stations_to

In [ ]:
all_stations = []
all_stations.extend([st.replace('ô', 'ő') for st in stations.index])
all_stations.extend([st.replace('ô', 'ő') for st in stations_to.index])
all_stations

In [ ]:
for i, name in enumerate(all_stations):
    if name[-1] == '-':
        all_stations[i] = name[:-1]

In [ ]:
all_stations = set(all_stations)

In [ ]:
stations = np.array(sorted(all_stations), dtype=np.str)
stations

I found out every station name can be only 20 character long

In [ ]:
len('Badacsonytördemic-Sz')

In [ ]:
num_stations = len(stations)
num_stations

In [ ]:
distances = np.zeros((num_stations, num_stations))
distances

In [ ]:
allomas1 = 10
allomas2 = 20
distances[allomas1, allomas2]


In [ ]:
stations[allomas1],stations[allomas2]

In [ ]:
index1 = np.where(stations == 'Badacsonylábdihegy')
index1[0][0]

In [ ]:
def get_index(station: str) -> int:
    return np.where(stations == station)[0][0]

I checked if my distances data containt every nighbouring stations.

In [ ]:
df_dist = pd.concat([df1, df2, df3], axis=0).reset_index(drop=True)
df_dist

In [ ]:
for index, row in df_dist.iterrows():
    for rem in ['mh.', 'mrh.', 'ipvk.', 'rh.','\xa0','','\r','0','ipvk', 'elá','pvh']:
        df_dist.loc[index,'Name'] = df_dist.loc[index,'Name'].replace(rem, '')

In [ ]:
matching_names = 0
for index, row in df_dist.iterrows():
    name = row['Name']
    if name in set(stations):
        matching_names += 1
    else:
        print(name)

In [ ]:
matching_names

In [ ]:
matching_stations = set([row['Name'] for index, row in df_dist.iterrows() if row['Name'] in stations])
matching_stations

In [ ]:
non_matching_stations = set([row['Name'] for index, row in df_dist.iterrows() if row['Name'] not in stations])
non_matching_stations

In [ ]:
len(matching_stations), len(non_matching_stations)

In [ ]:
dist_names = np.array(df_dist['Name'].values, dtype=np.str)

In [ ]:
missing_stations = set([station for station in stations if station not in dist_names ])
missing_stations

In [ ]:
len(missing_stations)

I built the distance matrix

In [ ]:
paths=[[]] # list of lists


d = 0
prev_not_null = None
prev = None
curr = None
for index, row in df_dist.iterrows():
    if index == 0: continue
    prev = df_dist.loc[index-1]['Name']
    curr = df_dist.loc[index]['Name']
    # dist = float(str(df_dist.loc[index]['Distance']).replace(',', '.'))
    dist = df_dist.loc[index]['Distance']

    if np.isnan(dist):
        prev = None
        prev_not_null = None
    # todo: ezt szedd ki mert akkor nem lesznek benne a hianyzo allomasok tavolsagai
#     if prev in missing_stations or curr in missing_stations:
#         continue


    if prev in stations:
        prev_not_null = prev
        d=dist # ha van (uj) elozo akkor mentsuk el a tavot
    else:
        if np.isnan(dist):  # new track
            d = 0
        else:  # prev station nem ismert de ugyanaz a track
            d += dist  # accumulate distance, will be zeroed out when we save it


    if curr not in stations:
#         print(f"{prev} {curr} {dist}")
        continue

    print(f"{prev_not_null} {curr} {d}")
    if not np.isnan(dist):
        if prev_not_null:
            distances[get_index(prev_not_null), get_index(curr)] = d
            distances[get_index(curr), get_index(prev_not_null)] = d
            paths[-1].append(curr)

        d=0

    else:  # new track
        paths.append([])
        paths[-1].append(curr)

In [ ]:
paths

In [ ]:
for path in paths:
    if len(path) <= 1: continue
    print(path)

In [ ]:
# Function to print all sub strings
def subString(Str,n):
    # Pick starting point
    for Len in range(3, n + 1):

        # Pick ending point
        for i in range(n - Len + 1):

            # Print characters from current 
            # starting point to current ending 
            # point.
            j = i + Len - 1

            yield [Str[k] for k in range(i, j + 1)]

In [ ]:
for path in paths:
    if len(path) == 1: continue
    for subpath in subString(path, len(path)):
        d = 0
        for i in range(len(subpath) - 1):
            d += distances[get_index(subpath[i]), get_index(subpath[i + 1])]
#             print(subpath[i], subpath[i + 1])
        distances[get_index(subpath[0]), get_index(subpath[-1])] = d
        distances[get_index(subpath[-1]), get_index(subpath[0])] = d
#     break

In [ ]:
agg = df.groupby(['From', 'To'])
agg = agg.count().reset_index()

In [ ]:
agg['dist'] = agg.apply(lambda row: distances[get_index(row['From'].replace('ô', 'ő')), get_index(row['To'].replace('ô', 'ő'))], axis=1)